In [2]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [3]:
import sys
sys.path.insert(0, '/workspaces/b2b-wf-experiments/src')

from components.data_evaluation_preprocessor import DataEvaluationPreprocessor
from components.data_evaluator import Evaluation

In [20]:
historical_work_hours = pd.read_csv('/workspaces/b2b-wf-experiments/src/notebook/historical_data.csv', index_col=False)
historical_work_hours['Appointment_Day'] = pd.to_datetime(historical_work_hours['Appointment_Day'])
historical_work_hours.sort_values(by=['series', 'Appointment_Day'], inplace=True)
historical_work_hours.head()

,Appointment_Day,Product_Grp,Work_Order_Action_Grp,District,Region_Type,SWT,series
0,2022-01-01,Managed,Install,AFF Abitibi,Tier 2,0.0,AFF Abitibi_Managed_Tier 2_Install
1,2022-01-02,Managed,Install,AFF Abitibi,Tier 2,0.0,AFF Abitibi_Managed_Tier 2_Install
2,2022-01-03,Managed,Install,AFF Abitibi,Tier 2,0.0,AFF Abitibi_Managed_Tier 2_Install
3,2022-01-04,Managed,Install,AFF Abitibi,Tier 2,0.0,AFF Abitibi_Managed_Tier 2_Install
4,2022-01-05,Managed,Install,AFF Abitibi,Tier 2,2.0,AFF Abitibi_Managed_Tier 2_Install


In [22]:
train_data = historical_work_hours[historical_work_hours['Appointment_Day'] < '2024-07-01']
test_data = historical_work_hours[historical_work_hours['Appointment_Day'] >= '2024-07-01']

In [32]:
# for each series in train_data, fit a simple exponential smoothing model
# then use the model to predict the next 184 days
# then plot the predicted values against the actual values

simple_exp_smoothing_predictions = pd.DataFrame()
for series in train_data['series'].unique():
    series_data = train_data[train_data['series'] == series]
    series_data.set_index('Appointment_Day', inplace=True)
    series_data = series_data.asfreq('D')


    model = SimpleExpSmoothing(series_data['SWT'], initialization_method='estimated')
    model_fit = model.fit()

    
    forecast = model_fit.predict(start=len(series_data), end=len(series_data) + 183)

    forecast_data = pd.DataFrame()
    forecast_data['SWT'] = forecast
    forecast_data['Appointment_Day'] = pd.date_range(
        start=test_data['Appointment_Day'].iloc[0], periods=184)
    simple_exp_smoothing_predictions = pd.concat([simple_exp_smoothing_predictions, forecast_data])

    district, product_grp, region_type, work_order_action_grp = series.split('_')

    simple_exp_smoothing_predictions['District'] = district
    simple_exp_smoothing_predictions['Product_Grp'] = product_grp
    simple_exp_smoothing_predictions['Region_Type'] = region_type
    simple_exp_smoothing_predictions['Work_Order_Action_Grp'] = work_order_action_grp
    simple_exp_smoothing_predictions['series'] = series

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
/opt/conda/lib/python3.10/site-packages/sta

In [33]:
historical_data = DataEvaluationPreprocessor(test_data.drop(columns=['series']))
forecast_data = DataEvaluationPreprocessor(simple_exp_smoothing_predictions.drop(columns=['series']))


evaluation = Evaluation(historical_data, forecast_data)

minimal_features_wape = {
    'overall': evaluation.calculate_metric('wape'),
    'Tier 1': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 1'}),
    'Tier 2': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 2'}),
    'Tier 3': evaluation.calculate_metric('wape', filters={'Region_Type': 'Tier 3'})
}

print(minimal_features_wape)

ValueError: Cannot set a DataFrame with multiple columns to the single column series_id